In [66]:
%matplotlib inline

# Imports
import re
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display
from sklearn import model_selection
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score)
from sklearn.naive_bayes import MultinomialNB
from wordcloud import STOPWORDS, WordCloud
import sys

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
stclass = pd.read_csv('/content/drive/MyDrive/DL_data/StudentClass.csv')
stclass.head(10)

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle
0,1.0,A,MW,15:30:00,9/1/2004,Math
1,1.0,A,M,19:00:00,9/1/2004,English
2,1.0,C,W,19:00:00,9/1/2004,Physics
3,1.0,B,MW,11:00:00,9/1/2004,Art
4,1.0,B,MW,12:30:00,9/1/2005,SocialStudy
5,1.0,B,TTH,12:30:00,9/1/2005,PE
6,1.0,B,M,19:00:00,9/1/2005,ForeignLanguage
7,1.0,B,MW,11:00:00,9/1/2005,Statistics
8,1.0,B,TU,16:00:00,9/1/2006,Music
9,1.0,C,M,16:00:00,3/1/2007,Drawing


In [69]:
stgrad = pd.read_csv('/content/drive/MyDrive/DL_data/StudentGraduate.csv')
stgrad

,ID,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,2,8,10,7,6,8,10,0,3.712993,1,558,0,MGMT
2,3,8,7,6,6,4,4,0,4.373719,1,634,1,MKTG
3,4,8,8,6,6,4,2,0,4.046570,0,386,0,MKTG
4,5,8,6,6,4,4,3,1,2.560076,0,381,0,MGMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,8,8,7,6,6,3,2,3.002898,1,441,0,MGMT
996,997,8,10,6,7,6,2,0,3.910183,0,443,1,ACCT
997,998,8,10,6,8,6,6,0,3.267001,0,515,0,MGMT
998,999,8,8,7,8,8,2,1,3.474561,0,459,0,MKTG


In [70]:
stclass.head()

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle
0,1.0,A,MW,15:30:00,9/1/2004,Math
1,1.0,A,M,19:00:00,9/1/2004,English
2,1.0,C,W,19:00:00,9/1/2004,Physics
3,1.0,B,MW,11:00:00,9/1/2004,Art
4,1.0,B,MW,12:30:00,9/1/2005,SocialStudy


In [71]:
stgrad.head()

,ID,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,2,8,10,7,6,8,10,0,3.712993,1,558,0,MGMT
2,3,8,7,6,6,4,4,0,4.373719,1,634,1,MKTG
3,4,8,8,6,6,4,2,0,4.046570,0,386,0,MKTG
4,5,8,6,6,4,4,3,1,2.560076,0,381,0,MGMT


In [72]:
print(stclass.columns)
print(stgrad.columns)


Index(['StudentID', 'GRADE', 'ClassWeekDay', 'ClassMeetTime', 'ClassDate',
       'ClassTitle'],
      dtype='object')
Index(['ID', 'HS_ENGLISH', 'HS_MATH', 'HS_HISTORY', 'HS_LAB_SCIENCES',
       'HS_FOREIGN_LANGUAGE', 'HS_ART', 'HS_ELECTIVES', 'HS_GPA', 'sex',
       'WRITINGScore', 'GraduateONTime', 'Major'],
      dtype='object')


#Quesiton 1: Do students perform better in morning class (before 12:00PM) or afternoon?

In [73]:
Studentclass=pd.read_csv('/content/drive/MyDrive/DL_data/StudentClass.csv')
StudentGraduate=pd.read_csv('/content/drive/MyDrive/DL_data/StudentGraduate.csv')

In [74]:
Studentclass_new=Studentclass.dropna(subset=['ClassMeetTime'])
#here we are splitting the dataset based on the classtime into Morning class and afternoon class
Studentclass_afternoon=Studentclass[Studentclass['ClassMeetTime']>'12:00:00']
Studentclass_morning=Studentclass[Studentclass['ClassMeetTime']<'12:00:00']
Studentclass_morning['GRADE'].count()

1612

In [75]:
count=0
for i, row in Studentclass_morning.iterrows():
    if row['GRADE']=='A':
        count=count+4
    elif row['GRADE']=='B':
        count=count+3
    elif row['GRADE']=='C':
        count=count+2
    elif row['GRADE']=='D':
        count=count+1
print(count/Studentclass_morning['GRADE'].count())

2.8138957816377173


In [76]:
count=0
for i, row in Studentclass_afternoon.iterrows():
    if row['GRADE']=='A':
        count=count+4
    elif row['GRADE']=='B':
        count=count+3
    elif row['GRADE']=='C':
        count=count+2
    elif row['GRADE']=='D':
        count=count+1
print(count/Studentclass_afternoon['GRADE'].count())

2.775314559616537


In [77]:
percent_morning=(2.8138957816377173/4)*100
print('percent of average gpa of student attending morning class ',end="")
print(percent_morning)
percent_afternoon=(2.775314559616537/4)*100
print('percent of average gpa of student attending afternoon class ',end="")
print(percent_afternoon)

percent of average gpa of student attending morning class 70.34739454094293
percent of average gpa of student attending afternoon class 69.38286399041343


#Question 2: Math, Physics and Statistics are considered quantitative classes. If a student take 2 or more quantitative classes in one semester, will he/she have lower average score?

In [78]:
import csv
d={}
grade = {}
points = {'A':4, 'B':3, 'C':2, 'D':1, 'F':0}
year_grades = {}
with open('/content/drive/MyDrive/DL_data/StudentClass.csv', mode='r') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        if line[0]=='':
            break
        if int(line[4].split('/')[2]) in year_grades:
            year_grades[int(line[4].split('/')[2])].append(points[line[1]])
        else:
            year_grades[int(line[4].split('/')[2])] = [points[line[1]]]
        if int(line[0]) in d:
            if int(line[4].split('/')[2]) in d[int(line[0])]:
                d[int(line[0])][int(line[4].split('/')[2])].append(line[5])
                grade[int(line[0])][int(line[4].split('/')[2])].append(line[1])
            else:
                d[int(line[0])][int(line[4].split('/')[2])] = [line[5]]
                grade[int(line[0])][int(line[4].split('/')[2])] = [line[1]]
        else:
            d[int(line[0])]={int(line[4].split('/')[2]):[line[5]]}
            grade[int(line[0])]={int(line[4].split('/')[2]):[line[1]]}
    for year in year_grades:
        year_grades[year] = sum(year_grades[year])/len(year_grades[year])
    stud_qclass_grade = {}
    for student in d:
        for year in d[student]:
            if ('Math' in d[student][year] and 'Physics' in d[student][year]) or ('Math' in d[student][year] and 'Statistics' in d[student][year]) or ('Statistics' in d[student][year] and 'Physics' in d[student][year]):
                s = 0
                for g in grade[student][year]:
                    s = s + points[g] 
                stud_qclass_grade[student] = {str(year)+' qclass': s/len(grade[student][year]), str(year): year_grades[year]}
    for std in stud_qclass_grade:
      print(str(std)+"--->"+str(stud_qclass_grade[std]))

1--->{'2004 qclass': 3.25, '2004': 2.7080557238037555}
2--->{'2006 qclass': 3.4, '2006': 2.78990990990991}
3--->{'2005 qclass': 3.5, '2005': 2.8646926536731634}
4--->{'2006 qclass': 3.4, '2006': 2.78990990990991}
6--->{'2006 qclass': 2.6666666666666665, '2006': 2.78990990990991}
7--->{'2005 qclass': 3.142857142857143, '2005': 2.8646926536731634}
9--->{'2004 qclass': 2.6666666666666665, '2004': 2.7080557238037555}
10--->{'2010 qclass': 3.4285714285714284, '2010': 2.5821596244131455}
11--->{'2006 qclass': 2.3333333333333335, '2006': 2.78990990990991}
12--->{'2004 qclass': 1.3333333333333333, '2004': 2.7080557238037555}
13--->{'2004 qclass': 3.5, '2004': 2.7080557238037555}
14--->{'2005 qclass': 2.2222222222222223, '2005': 2.8646926536731634}
15--->{'2004 qclass': 3.0, '2004': 2.7080557238037555}
16--->{'2010 qclass': 3.142857142857143, '2010': 2.5821596244131455}
17--->{'2011 qclass': 2.25, '2011': 2.4146341463414633}
19--->{'2004 qclass': 3.0, '2004': 2.7080557238037555}
20--->{'2005 qc

In [82]:
gre = []
less = []
for std in stud_qclass_grade:
  if stud_qclass_grade[std][list(stud_qclass_grade[std].keys())[0]] >= stud_qclass_grade[std][list(stud_qclass_grade[std].keys())[1]]:
    gre.append(stud_qclass_grade[std][list(stud_qclass_grade[std].keys())[0]])
  else:
    less.append(stud_qclass_grade[std][list(stud_qclass_grade[std].keys())[0]])
print('Number of student having higher quantitative GPA ',end="")
print(len(gre))
print('Number of student having lower quantitative GPA ',end="")

print(len(less)) 

Number of student having higher quantitative GPA 427
Number of student having lower quantitative GPA 424


#Question 3: Which Major has higher quantitative class GPA?

In [83]:
maj = {}
stud_sub = {}
maj_qclass = {}
with open('/content/drive/MyDrive/DL_data/StudentClass.csv', mode='r') as f2:
  csvreader = csv.reader(f2)
  next(csvreader)
  for line in csvreader:
    if line[0]=='':
      break
    if int(line[0]) in stud_sub:
      stud_sub[int(line[0])][line[5]] = line[1]
    else:
      stud_sub[int(line[0])] = {line[5]:line[1]}
with open('/content/drive/MyDrive/DL_data/StudentGraduate.csv', mode='r') as f1:
  csvreader = csv.reader(f1)
  next(csvreader)
  for line in csvreader:
    if line[0]=='':
      break
    if line[12] in maj:
      maj[line[12]].append(int(line[0]))
    else:
      maj[line[12]] = [int(line[0])]
  for major in maj:
    math = []
    phy = []
    stat = []
    for id in maj[major]:
      if 'Math' in stud_sub[id]:
        math.append(stud_sub[id]['Math'])
      if 'Physics' in stud_sub[id]:
        phy.append(stud_sub[id]['Physics'])
      if 'Statistics' in stud_sub[id]:
        stat.append(stud_sub[id]['Statistics'])
    math_sum = 0
    phy_sum = 0
    stat_sum = 0
    for g in math:
      math_sum = math_sum + points[g]
    math_avg = math_sum/len(math)
    for g in phy:
      phy_sum = phy_sum + points[g]
    phy_avg = phy_sum/len(phy)
    for g in stat:
      stat_sum = stat_sum + points[g]
    stat_avg = stat_sum/len(stat)
    maj_qclass[major] = {'Math':math_avg, 'Physics':phy_avg, 'Statistics':stat_avg}
  for maj in maj_qclass:
    print(str(maj)+"---->"+str(maj_qclass[maj]))

FIN---->{'Math': 2.8487394957983194, 'Physics': 2.808333333333333, 'Statistics': 2.715481171548117}
MGMT---->{'Math': 2.7655677655677655, 'Physics': 2.854014598540146, 'Statistics': 2.767272727272727}
MKTG---->{'Math': 2.8227848101265822, 'Physics': 2.8649789029535864, 'Statistics': 2.7983193277310923}
ACCT---->{'Math': 2.772972972972973, 'Physics': 2.7621621621621624, 'Statistics': 2.777173913043478}
I S---->{'Math': 2.6595744680851063, 'Physics': 2.8627450980392157, 'Statistics': 2.84}


In [84]:
sum_maj = {}
for maj in maj_qclass:
  #print(str(maj)+"---->"+str(sum(list(maj_qclass[maj].values()))))
  sum_maj[maj]=sum(list(maj_qclass[maj].values()))
#print(sum_maj)
keys = list(sum_maj.keys())
values = list(sum_maj.values())
sorted_value_index = np.argsort(values)
sorted_dict = {keys[i]: values[i] for i in sorted_value_index}
for maj in sorted_dict:
  print(str(maj)+"---->"+str(sorted_dict[maj]))
#print(sorted_dict)

ACCT---->8.312309048178614
I S---->8.362319566124322
FIN---->8.37255400067977
MGMT---->8.386855091380639
MKTG---->8.48608304081126
